# LangGraph Parallel Tool Execution

Understanding how LangGraph executes multiple independent tools simultaneously.

## Learning Objectives

By the end of this notebook, you will:

1. **Understand parallel execution** - When tasks are independent, the LLM calls multiple tools in a single request
2. **Recognize the parallel pattern** - A single AIMessage contains multiple tool_calls that execute simultaneously
3. **Verify parallel execution** - Examine message count and tool_calls structure to confirm parallel behavior

## 1. Environment Setup

In [ ]:
# Core imports
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv
from typing import Literal

load_dotenv("../../.env")
print("✅ Environment loaded")

## 2. Define Tools

In [ ]:
# Define tools
@tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Convert currency from one type to another.
    
    Use this tool when users need to convert monetary amounts between
    different currencies. Supports USD, EUR, GBP, INR, and JPY.
    """
    exchange_rates = {"USD": 1.0, "EUR": 0.92, "GBP": 0.79, "INR": 83.12, "JPY": 149.50}
    from_currency = from_currency.upper()
    to_currency = to_currency.upper()
    
    if from_currency not in exchange_rates or to_currency not in exchange_rates:
        return f"Error: Unsupported currency"
    
    amount_in_usd = amount / exchange_rates[from_currency]
    converted_amount = amount_in_usd * exchange_rates[to_currency]
    effective_rate = exchange_rates[to_currency] / exchange_rates[from_currency]
    
    return (
        f"Conversion Result:\n"
        f"  {amount:,.2f} {from_currency} = {converted_amount:,.2f} {to_currency}\n"
        f"  Exchange Rate: 1 {from_currency} = {effective_rate:.4f} {to_currency}"
    )

@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.
    
    Use this tool when users want to know their monthly loan payment,
    total repayment amount, or total interest for a loan.
    """
    if principal <= 0 or annual_interest_rate < 0 or tenure_months <= 0:
        return "Error: Invalid input parameters"
    
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    return (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months\n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )

print("✅ Tools defined")

## 3. Initialize LLM and Build Graph

In [ ]:
# Initialize LLM and build graph
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=1024
)

tools = [currency_converter, emi_calculator]
llm_with_tools = llm.bind_tools(tools)

def call_llm(state: MessagesState):
    """LLM node: Calls LLM with current messages."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Router: Check if agent wants to use tools."""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

# Build graph
workflow = StateGraph(MessagesState)
workflow.add_node("llm", call_llm)
workflow.add_node("tools", ToolNode(tools))
workflow.add_edge(START, "llm")
workflow.add_conditional_edges("llm", should_continue, {"tools": "tools", END: END})
workflow.add_edge("tools", "llm")

app = workflow.compile()
print("✅ Graph compiled")

## 4. Test Parallel Execution

Query with **independent tasks** using "AND ALSO" to signal no dependencies between operations.

In [ ]:
# Test parallel execution: Independent tasks
state = {
    "messages": [
        HumanMessage(content="Convert 100000 USD to EUR AND ALSO calculate EMI for 500000 INR at 8.5% for 60 months")
    ]
}

result = app.invoke(state)
print(f"Total messages: {len(result['messages'])}")

### Verify Parallel Tool Calls

Check if multiple tools were called in a **single AIMessage**.

In [ ]:
# Verify parallel tool calls - check for multiple tool_calls in single AIMessage
tool_call_message = result['messages'][1]

print(f"Number of tool_calls: {len(tool_call_message.tool_calls)}")

if len(tool_call_message.tool_calls) > 1:
    print(f"\n🚀 CONFIRMED: Parallel execution!")
    print(f"   {len(tool_call_message.tool_calls)} tools called in a SINGLE AIMessage")

print("\nTool Calls:")
for i, tc in enumerate(tool_call_message.tool_calls, 1):
    print(f"\n  Tool {i}:")
    print(f"    Name: {tc['name']}")
    print(f"    Args: {tc['args']}")

### Examine Complete Messages

Look at all messages to understand the parallel execution flow.

In [ ]:
result['messages']

In [ ]:
dict(result['messages'][0])

In [ ]:
dict(result['messages'][1])

In [ ]:
dict(result['messages'][2])

In [ ]:
dict(result['messages'][3])

In [ ]:
dict(result['messages'][4])

### Message Flow Visualization

In [ ]:
# Examine complete message flow
print("MESSAGE FLOW:\n" + "=" * 70)

for i, msg in enumerate(result["messages"], 1):
    print(f"\n[{i}] {type(msg).__name__}")
    
    if isinstance(msg, HumanMessage):
        print(f"    Content: {msg.content[:60]}...")
    elif isinstance(msg, AIMessage):
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            print(f"    Requesting {len(msg.tool_calls)} tool(s) IN PARALLEL")
            for tc in msg.tool_calls:
                print(f"      • {tc['name']}")
        else:
            print(f"    Final response: {msg.content[:80]}...")
    elif isinstance(msg, ToolMessage):
        first_line = msg.content.split('\n')[0]
        print(f"    Result: {first_line}")

### Final Response

In [ ]:
# Show final response
print("FINAL RESPONSE:\n" + "=" * 70)
print(result["messages"][-1].content)

## 5. Streaming Execution

Observe the parallel execution in real-time through streaming.

In [ ]:
# Stream execution to see real-time flow
state_stream = {
    "messages": [
        HumanMessage(content="Convert 100000 USD to EUR AND ALSO calculate EMI for 500000 INR at 8.5% for 60 months")
    ]
}

print("STREAMING EXECUTION\n" + "=" * 70)
step_count = 0

for event in app.stream(state_stream):
    for node_name, data in event.items():
        step_count += 1
        print(f"\n[Step {step_count}] Node: '{node_name}'")
        
        if "messages" in data:
            for msg in data["messages"]:
                if isinstance(msg, AIMessage):
                    if hasattr(msg, "tool_calls") and msg.tool_calls:
                        print(f"  🚀 PARALLEL CALL: {len(msg.tool_calls)} tools requested")
                        for tc in msg.tool_calls:
                            print(f"     • {tc['name']}")
                    else:
                        print(f"  💬 Final response generated")
                elif isinstance(msg, ToolMessage):
                    print(f"  ✅ Tool executed")

print(f"\nTotal steps: {step_count}")

## Conclusion

In this notebook, you learned:

✅ **Parallel execution** - When tasks are independent (no dependencies), the LLM calls multiple tools simultaneously in a single request, making the workflow more efficient

✅ **Parallel pattern** - A single AIMessage contains multiple tool_calls (e.g., `tool_calls: [currency_converter, emi_calculator]`), resulting in 5 total messages instead of 6+ for sequential execution

✅ **Verification** - Check `len(tool_calls)` in the AIMessage: if > 1, it's parallel execution; you'll see multiple ToolMessages returned before the final synthesis

### Next Steps

Next, we'll explore **sequential tool execution** where tasks have dependencies and must be executed one after another.